In [ ]:
%pip install easyocr folium==0.2.1 aiogram nest-asyncio

In [ ]:
%env TOKEN=your_telegram_bot_token

In [6]:
import nest_asyncio
nest_asyncio.apply()

import easyocr
import os
from aiogram import Bot, Dispatcher, executor, types

token = os.environ['TOKEN']
bot = Bot(token=token)
dp = Dispatcher(bot)


def text_recog(in_file_path):
    reader = easyocr.Reader(["uk", "en", "ru"], gpu=True)
    result = reader.readtext(in_file_path, detail=0, paragraph=True)
    return result


@dp.message_handler(commands="start")
async def start(message: types.Message):
    await message.answer("Take a photo and i'll return text to you")


@dp.message_handler(content_types=types.ContentType.PHOTO)
async def process_image(message: types.Message):
  try:
    await message.answer('Taking photo...')
    photo = message.photo[-1].file_id
    file = await bot.get_file(photo)
    file_path = file.file_path
    await bot.download_file(file_path, "image.png")
    await message.answer("Recognizing...wait please")
    await message.answer("\n".join(text_recog("image.png")))
  except:
    await message.answer("Oops...error occured, try once more")

In [ ]:
if __name__ == '__main__':
    executor.start_polling(dp)